In [1]:
import numpy as np
import pandas as pd
import h5py
import glob
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
%matplotlib inline

In [ ]:
# Aggregate all X
with h5py.File("data/X/X_A.hdf5","r") as f:
    X = f["mfcc"][:]
    artist = [i.decode("utf-8") for i in f["artist"][:]]
    title = [i.decode("utf-8") for i in f["title"][:]]
    song_id = [i.decode("utf-8") for i in f["song_id"][:]]
        
files = glob.glob("data/X/X_*.hdf5")
files.remove("data/X/X_A.hdf5")

for file in files:
    with h5py.File(file,"r") as f:
        X = np.concatenate((X,f["mfcc"][:]))
        artist += [i.decode("utf-8") for i in f["artist"][:]]
        title += [i.decode("utf-8") for i in f["title"][:]]
        song_id += [i.decode("utf-8") for i in f["song_id"][:]]

In [ ]:
# Store aggregated X
with h5py.File("data/X/X.hdf5", "w") as f:
    f.create_dataset("mfcc", data=X)
    ascii_artist = [n.encode("ascii", "ignore") for n in artist]
    f.create_dataset('artist', data=ascii_artist)
    ascii_title = [n.encode("ascii", "ignore") for n in title]
    f.create_dataset('title', data=ascii_title)
    ascii_song_id = [n.encode("ascii", "ignore") for n in song_id]
    f.create_dataset('song_id', data=ascii_song_id)

In [ ]:
# Load X
with h5py.File("data/X/X.hdf5","r") as f:
    X = f["mfcc"][:]
    artist = [i.decode("utf-8") for i in f["artist"][:]]
    title = [i.decode("utf-8") for i in f["title"][:]]
    song_id = [i.decode("utf-8") for i in f["song_id"][:]]

In [ ]:
# Load Y
f = h5py.File("data/Y/U.hdf5", "r")
Y_temp = f["Y"][:]
songs = [i[0].decode("utf-8") for i in f["songs"][:]]
f.close()

In [ ]:
# Match X and Y
indices = [songs.index(song) for song in song_id]

Y = Y_temp[indices] 
songz = []
for i in indices:
    songz.append(songs[i])

In [ ]:
# Store data
with h5py.File("data/data.hdf5", "w") as f:
    f.create_dataset("X", data=X)
    ascii_artist = [n.encode("ascii", "ignore") for n in artist]
    f.create_dataset('artist', data=ascii_artist)
    ascii_title = [n.encode("ascii", "ignore") for n in title]
    f.create_dataset('song', data=ascii_title)
    f.create_dataset("Y", data=Y)

In [2]:
# Load data
with h5py.File("data/data.hdf5", "r") as f:
    X = f["X"][:]
    artist = f["artist"][:]
    song = f["song"][:]
    Y = f["Y"][:]

In [8]:
# Collaborative filtering sanity check
Y_1 = [i[9] for i in Y]
data = pd.DataFrame(data=[artist,song,Y_1]).T
data.columns = ["artist","song","latent"]
data.sort_values("latent",ascending=True)

,artist,song,latent
284888,b'Edward Sharpe & The Magnetic Zeros',b'Home',-0.43649
282972,b'The Black Keys',b'Tighten Up',-0.409518
321772,b'Bon Jovi',"b""Livin' On A Prayer""",-0.387138
78989,b'Simon Harris',b'Sample Track 2',-0.376182
214762,b'Eagles',b'Hotel California',-0.368227
94055,b'Florence + The Machine',b'Dog Days Are Over (Radio Edit)',-0.356047
370265,b'Radiohead',b'Creep (Explicit)',-0.355456
267962,b'Justin Bieber',b'Never Let You Go',-0.346028
65013,b'Taylor Swift',b'Love Story',-0.338601
374917,b'Taylor Swift',b'Love Story',-0.338601


In [ ]:
# Explore data
latent = [i[2] for i in Y]
extremes = [i for i in latent if i > 0.01 or i < -0.01]
print("Num extremes: "+str(len(extreme)))
print("Max: "+str(max(latent)))
print("Max: "+str(min(latent)))
plt.hist(latent);

In [ ]:
mfcc_features = [i[100][11] for i in X]
plt.hist(mfcc_feature);